<hr>

# Universidad Nacional de Colombia
## Clasificación y Reconocimiento de Patrones
### 2019

<hr>

# 2.1. Feature Selection

<hr>

## 2.1.2. Statistical Approach

A simple statistical approach consists in removing features with low variance, that is, features with low information.

<hr>

### Variance Threshold

In [ ]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold

In [ ]:
X = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]])

selector = VarianceThreshold(threshold=0.2)
X_new = selector.fit_transform(X)

In [ ]:
print("Original features: ",X)
print("Selected features: ",X_new)

<hr>
Try same approach with a known dataset: <a href="https://es.wikipedia.org/wiki/Iris_flor_conjunto_de_datos">Iris Dataset</a>

In [ ]:
!pip install tabulate

In [ ]:
from sklearn.datasets import load_iris
from tabulate import tabulate

In [ ]:
iris = load_iris()

X, y = iris.data, iris.target

In [ ]:
headers = ["feature 1", "feature 2", "feature 3", "feature 4"]
table = tabulate(X, headers, tablefmt="fancy_grid")
print(table)

In [ ]:
selector = VarianceThreshold(threshold=0.6) #feature selection
X_new = selector.fit_transform(X)

sel_features = np.squeeze(np.where(selector.get_support()==True)) #Where selection is True?

print("Features selected: ", sel_features)

In [ ]:
headers = ["feature "+str(i+1) for i in sel_features]
table = tabulate(X_new, headers, tablefmt="fancy_grid")
print(table)

In [ ]:
import matplotlib.pyplot as plt

colors = ['navy', 'turquoise', 'darkorange']

target_names = iris.target_names

color = [colors[i] for i in y]
label = [target_names[i] for i in y]

plt.figure(figsize=(5,5))
plt.title("Selected features using Variance Threshold")

plt.scatter(X_new[:, 0], X_new[:, 1], color=color)

plt.xlabel("feature "+str(sel_features[0]+1))
plt.ylabel("feature "+str(sel_features[1]+1))
plt.show()

<hr>

### Statistical Test $\chi^2$

We can use statistcal tests like chi-squared $\chi^2$ for features selection. This tests will answer the question:

<br>

<center> <font size=5> How independent are features from target? </font></center>

In [ ]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt

In [ ]:
iris = load_iris()

X, y = iris.data, iris.target

k=2 #best 2 independets features

selector = SelectKBest(score_func=chi2, k=2)

X_new = selector.fit_transform(X, y)

sel_features = np.squeeze(np.where(selector.get_support()==True)) #Where selection is True?

headers = ["feature "+str(i+1) for i in sel_features]
table = tabulate(X_new, headers, tablefmt="fancy_grid")
print(table)

In [ ]:
colors = ['navy', 'turquoise', 'darkorange']

target_names = iris.target_names

color = [colors[i] for i in y]
label = [target_names[i] for i in y]

plt.figure(figsize=(5,5))
plt.title("Selected features using Variance Threshold")

plt.scatter(X_new[:, 0], X_new[:, 1], color=color)

plt.xlabel("feature "+str(sel_features[0]+1))
plt.ylabel("feature "+str(sel_features[1]+1))
plt.show()

<hr>

## 2.1.3. Wrappers and Embedded Models

<hr>

### Recursive Feature Elimination (Wrappers)

Using and external predictor, executes k iterations to recursively remove non-relevant features until a number of features defined by user is reached.

<br>

<center><font size=3> <b>Main feature</b>: Uses iterations to reach a limit of features defined by user.</font></center>

In [ ]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

# Load the digits dataset
digits = load_digits()
X = digits.images.reshape((len(digits.images), -1))
y = digits.target

In [ ]:
index = 234
img = np.reshape(X[index,:], (8,8))
label = y[index]
# Plot pixel ranking
plt.figure(figsize=(4,4))
plt.imshow(img, cmap='gray')
plt.title("A sample from digits dataset with label "+str(label))
plt.show()

In [ ]:
# Create the RFE object and rank each pixel
model = LogisticRegression()
rfe = RFE(estimator=model, n_features_to_select=32, step=1) #best 32 features
rfe.fit(X, y)
ranking = rfe.ranking_.reshape(digits.images[0].shape)

In [ ]:
# Show ranked features
print rfe.ranking_

# Plot pixel ranking
plt.figure(figsize=(6,6))
plt.imshow(ranking, cmap=plt.cm.Blues)
plt.colorbar()
plt.title("Ranking of pixels with RFE - low is best", fontsize=10)
plt.show()

<hr>

### SelectFromModel (Embedded Models)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectFromModel

In [ ]:
#load digits dataset
digits = load_digits()
X = digits.images.reshape((len(digits.images), -1))
y = digits.target

#train prediction model
predictor = ExtraTreesClassifier()
predictor.fit(X, y)

In [ ]:
model = SelectFromModel(predictor, threshold=0.02, prefit=True)
X_new = model.transform(X) #dataset with selected features

In [ ]:
selection = model.get_support()
selection_map = np.reshape(selection, (8,8))

num_selected_features = np.count_nonzero(selection)

In [ ]:
# Plot pixel ranking
plt.figure(figsize=(6,6))
plt.imshow(selection_map, cmap='gray')
plt.title("Selection of "+str(num_selected_features)+" most important features using SfM", fontsize=10)
plt.show()

<hr>

# Feature Extraction

<hr>

## 2.2.2. Principal Component Analysis (PCA)

PCA consists in projecting a set of data over an hyperspace composed by principal dimensions of data.

<hr>

### PCA Using Single Value Decomposition

Eigenvalues can be extracted using a matrix factorization approach, for example, SVD. SVD is expressed as:

<br>

$X = U \cdot \sum \cdot V^T$

Where $V$ is a the matrix containing the eigenvalues of $X$

$
\mathbf{V} =
\begin{pmatrix}
  \mid & \mid & & \mid \\
  \mathbf{c_1} & \mathbf{c_2} & \cdots & \mathbf{c_n} \\
  \mid & \mid & & \mid
\end{pmatrix}
$

In [ ]:
#generate a test dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2])

ax.set_title("Input dataset")
ax.set_xlabel(r'$f_0$', fontsize=15)
ax.set_ylabel(r'$f_1$', fontsize=15)
ax.set_zlabel(r'$f_2$', fontsize=15)

In [ ]:
from numpy.linalg import svd

# Apply Single Value Decomposition over X
X_centered = X - X.mean(axis=0)
U, s, Vt = svd(X_centered) 

In [ ]:
print(Vt) #print Eigenvalues

In [ ]:
#Project X over first principal components

V_prime = Vt.T[:, :2]
X2D = np.dot(X_centered, V_prime)

In [ ]:
plt.title("Reduced Dataset")
plt.scatter(X2D[:,0], X2D[:,1])

plt.xlabel(r'$\lambda^0$')
plt.ylabel(r'$\lambda^1$')
plt.show()

<hr>

### PCA Using Scikit

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

In [ ]:
print("First 2 components by SVD: ",Vt[:2])
print("First 2 components by Scikit: ", pca.components_)

Apply PCA over iris dataset

In [ ]:
iris = load_iris()
X, y = iris.data, iris.target
k=2 #first 2 principal components

pca = PCA(n_components=k)
X_new = pca.fit_transform(X)

In [ ]:
colors = ['navy', 'turquoise', 'darkorange']

target_names = iris.target_names

color = [colors[i] for i in y]
label = [target_names[i] for i in y]

plt.figure(figsize=(5,5))
plt.title("Iris dataset projection over first 2 PC")

plt.scatter(X_new[:, 0], X_new[:, 1], color=color)

plt.xlabel(r'$\lambda_0$', fontsize=15)
plt.ylabel(r'$\lambda_1$', fontsize=15)
plt.show()

<hr>

### Non-Linear (PCA) / Kernel PCA

In most complex cases, we can use a mathematical non-linear function or kernel to perform complex transformations of data.

In [ ]:
from sklearn.datasets import make_circles

In [ ]:
np.random.seed(0)
X, y = make_circles(n_samples=400, factor=.3, noise=.05)

In [ ]:
colors = ['navy', 'darkorange']

target_names = iris.target_names

color = [colors[i] for i in y]
label = [target_names[i] for i in y]

plt.figure(figsize=(5,5))
plt.title("Circle dataset")

plt.scatter(X[:, 0], X[:, 1], color=color)

plt.xlabel(r'$f_0$', fontsize=15)
plt.ylabel(r'$f_1$', fontsize=15)
plt.show()

Use simple PCA and evaluate result

In [ ]:
pca = PCA(n_components=1)
X_new = pca.fit_transform(X)

In [ ]:
colors = ['navy', 'darkorange']

target_names = iris.target_names

color = [colors[i] for i in y]
label = [target_names[i] for i in y]

plt.figure(figsize=(5,5))
plt.title("Circle dataset")

plt.scatter(np.linspace(0,len(X_new),len(X_new)), X_new, color=color)

plt.ylabel(r'$\lambda$', fontsize=15)
plt.show()

Now try with kernel PCA

In [ ]:
from sklearn.decomposition import KernelPCA

kpca = KernelPCA(kernel="rbf", fit_inverse_transform=True, gamma=10)
X_kpca = kpca.fit_transform(X)

In [ ]:
colors = ['navy', 'darkorange']

color = [colors[i] for i in y]

plt.figure(figsize=(5,5))
plt.title("Circle dataset")

plt.scatter(X_kpca[:, 0], X_kpca[:, 1], color=color)

plt.xlabel(r'$\lambda_0$', fontsize=15)
plt.ylabel(r'$\lambda_1$', fontsize=15)
plt.show()

<hr>

# 4. Classwork

Work over next problems, using previous code as a base:

<hr>

## 1. Load diabetes regression dataset 

using __load_diabetes()__ from sklearn.datasets
 1. Tabulate data X and y.
 2. Apply a feature selection approach to obtain best 3 features.
 3. Plot using __plt.plot()__ 3 plots: $(x0, y), (x1, y), (x2, y)$

In [ ]:
from sklearn.datasets import load_diabetes

dataset = load_diabetes()
X, y = dataset.data, dataset.target

#normalize data (center and spread)
X = X - np.mean(X, axis=0)
X = X / np.std(X, axis=0)

In [ ]:
headers = ["feat"+str(i) for i in range(1,11)]
table = tabulate(X, headers, tablefmt="fancy_grid", floatfmt=".2f", numalign='center')
print table

In [ ]:
from sklearn.feature_selection import f_regression

#debido a que este problema es de regresion, utilizamos la funcion de score f_regression
selector = SelectKBest(score_func=f_regression, k=3)
X_new = selector.fit_transform(X, y)
sel_features = np.squeeze(np.where(selector.get_support()==True)) #Where selection is True?

headers = ["feat"+str(i+1) for i in sel_features]
table = tabulate(X_new, headers, tablefmt="fancy_grid",floatfmt=".2f", numalign='center')
print(table)

In [ ]:
plt.figure(figsize=(10,10))

plt.ylabel("Diabetes index")
plt.xlabel("Features")

scale = 1

plt.plot(X[:,sel_features[0]]*scale, y, 'r*')
plt.plot(X[:,sel_features[1]]*scale, y, 'g*')
plt.plot(X[:,sel_features[2]]*scale, y, 'b*')
plt.show()

<hr>

## 2.  Load breast_cancer classification dataset 

using __load_breast_cancer()__ from sklearn.datasets: http://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+%28diagnostic%29
 1. Tabulate data X and y.
 2. Apply a feature extraction approach to obtain 2 features.
 3. Plot using __plt.scatter()__ to obtain a figure where can be observed elements and classes.

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
dataset = load_breast_cancer()
X, y = dataset.data, dataset.target

In [ ]:
#Debido a que la cantidad de caracteristicas es muy grande, tabulamos solo una porcion del dataset
headers = ["feat"+str(i) for i in range(1,11)]
table = tabulate(X[:,0:10], headers, tablefmt="fancy_grid", floatfmt=".2f", numalign='center')
print table

In [ ]:
from sklearn.decomposition import PCA

#normalize data (center and spread)
X = X - np.mean(X, axis=0)
X = X / np.std(X, axis=0)

#apply PCA
pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

In [ ]:
#Plot results using extracted features and two classes

colors = ['navy', 'darkorange']

target_names = dataset.target_names

color = [colors[i] for i in y]
label = [target_names[i] for i in y]

plt.figure(figsize=(7,7))
plt.title("Breast cancer dataset projection over first 2 PC")

plt.scatter(X2D[:, 0], X2D[:, 1], color=color)

plt.xlabel(r'$\lambda_0$', fontsize=15)
plt.ylabel(r'$\lambda_1$', fontsize=15)
plt.show()